# 🤵 Примеры работы классических *unsupervised* методов обнаружения аномалий

### 🌐 Установка [pyod](https://github.com/yzhao062/pyod)

In [ ]:
!pip3 install pyod

## ⚙️ Загрузка данных

In [4]:
import json

c = 0.1  # отношение количества аномальных экземпляров к нормальным

# Загрузка предобработанных текстов
with open('my_dict.json', 'r') as fp:
    topics_dict = json.load(fp)
    
t1 = 'Наука и техника-Оружие'
t2 = 'Экономика-Госэкономика'
    
normal_data = topics_dict[t1]
anomal_data = topics_dict[t2][:min(int(c * len(normal_data)) + 1, len(topics_dict[t2]))]
all_data = normal_data + anomal_data

print("Количество нормальных экземпляров = {}".format(len(normal_data)))
print("Количество аномальных экземпляров = {}".format(len(anomal_data)))

Количество нормальных экземпляров = 1000
Количество аномальных экземпляров = 101


## 💅 Предобработка данных

In [ ]:
# from gensim.parsing.preprocessing import strip_tags
# from gensim.parsing.preprocessing import strip_numeric
# from gensim.parsing.preprocessing import strip_punctuation
# from gensim.parsing.preprocessing import strip_non_alphanum
# from gensim.parsing.preprocessing import strip_short
# from gensim.parsing.preprocessing import strip_multiple_whitespaces

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

# import re
# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()


# def preprocess_text(text, tokenization=False):
#     """
#     Предобрабатывает исходный русский текст.
#     Выполняет очистку, токенизацию и лемматизацию.
#     Возвращает список токенов или предобработанный текст.
#     """
#     text = text.lower()                     # приведение к нижнему регистру
#     text = strip_tags(text)                 # удаление тэгов
#     text = strip_numeric(text)              # удаление цифр
#     text = strip_punctuation(text)          # удаление пунктуации
#     text = strip_non_alphanum(text)         # удаление небуквенных символов
#     text = strip_short(text)                # удаление коротких слов
#     text = strip_multiple_whitespaces(text) # удаление множественных пробелов
    
#     # Токенизация
#     list_of_words = word_tokenize(text, language='russian') 

#     # Удаление стоп-слов
#     list_of_words = [w for w in list_of_words if w not in stopwords.words('russian')]

#     # Лемматизация
#     for i in range(len(list_of_words)): 
#         list_of_words[i] = morph.parse(list_of_words[i])[0].normal_form

#     if tokenization: # возвращаем список слов
#         return list_of_words
#     else: # возвращаем строку
#         return ' '.join(w for w in list_of_words) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# normal_data = [preprocess_text(text) for text in normal_data]
# anomal_data = [preprocess_text(text) for text in anomal_data]
# all_data = normal_data + anomal_data

## 🎰 TF-IDF векторизация

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
all_data_tf = vectorizer.fit_transform(all_data).toarray()

print(all_data_tf.shape)

(1101, 11427)


## 📝 Universal Sentence Encoder (USE)

In [ ]:
# import tensorflow as tf
# import tensorflow_hub as hub
# hub_layer = hub.KerasLayer(
#     'https://tfhub.dev/google/universal-sentence-encoder/4',
#     input_shape=[], 
#     dtype=tf.string,
#     trainable=False)

## 🤹‍♂️ Формирование выборок



In [6]:
from sklearn.utils import shuffle
import numpy as np

x = all_data_tf
# x_use = (hub_layer(all_data)).numpy()
y = np.array([False] * len(normal_data) + [True] * len(anomal_data))

all_data, x, y = shuffle(all_data, x, y, random_state=123)
print("Всего экземпляров = {}".format(len(all_data)))
print("(Кол-во текстов, число признаков текста) = {}".format(x.shape))
print("Кол-во меток = {}".format(len(y)))
print("Кол-во нормальных экземпляров = {}".format(len(normal_data)))
print("Кол-во аномальных экземпляров = {}".format(len(anomal_data)))

Всего экземпляров = 1101
(Кол-во текстов, число признаков текста) = (1101, 11427)
Кол-во меток = 1101
Кол-во нормальных экземпляров = 1000
Кол-во аномальных экземпляров = 101


## 1️⃣ 🌲 Isolation Forest (IF, Изолирующий лес)
У меня на практике 5000 деревьев дают такой же результат, что и 15000.

### 🅰️ Метод из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score

# Задаем модель
clf = IsolationForest(
    contamination=0.1, # отношение аномалий к выборке
    n_estimators=5000, # кол-во деревьев в ансамбле
    max_samples=0.7,
    max_features=0.7,
    random_state=123, 
    bootstrap=True,
    n_jobs=-1, # задействем все процессы в распоряжении
)

# Тренировка модели
clf.fit(x)

# Предсказание модели
y_predict_iforest = clf.predict(x)

# Переименовываем метки в соответствии с нашими
y_predict_iforest[y_predict_iforest == 1] = False
y_predict_iforest[y_predict_iforest == -1] = True

# Считаем метрику ROC AUC
auc_iforest = roc_auc_score(y, y_predict_iforest)
print("auc_iforest = ", auc_iforest)

### 🅱️ Метод из [pyod](https://github.com/yzhao062/pyod)
Ответы показывает такие же, что и в sclearn, так как интерпретация на основе их реализации.

####  🎰  TF-IDF

In [21]:
### TF-IDF ###

import pyod
from pyod.models import iforest
from sklearn.metrics import roc_auc_score

# Задаем модель
iforest_clf = iforest.IForest(
    contamination=0.1,
    n_estimators=5000,
    max_samples=0.7,
    max_features=0.7,
    bootstrap=True,
    random_state=123,
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_iforest = iforest_clf.fit_predict(x)

# Считаем метрику ROC AUC
# auc_iforest = roc_auc_score(y, y_predict_iforest)
auc_iforest = iforest_clf.fit_predict_score(x, y)
print("auc_iforest = ", auc_iforest)

/Users/pavelmamaev/Desktop/Sait/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


KeyboardInterrupt: 

#### 📝 Universal Sentence Encoder

In [11]:
### Universal Sentence Encoder ###

import pyod
from pyod.models import iforest
from sklearn.metrics import roc_auc_score

# Задаем модель
iforest_clf = iforest.IForest(
    contamination=0.1,
    n_estimators=5000,
    max_samples=1.0,
    bootstrap=True,
    random_state=123,
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_iforest = iforest_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_iforest = roc_auc_score(y, y_predict_iforest)
print("auc_iforest = ", auc_iforest)

NameError: name 'x_use' is not defined

## 2️⃣ 🧮 Local Outlier Factor (LOF, Локальный уровень выброса)

###  🎰  TF-IDF

In [10]:
import pyod
from pyod.models import lof
from sklearn.metrics import roc_auc_score


# Создание модели
lof_clf = lof.LOF(
    contamination=0.1,
    n_neighbors=3,
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_lof = lof_clf.fit_predict(x)
# auc_lof = roc_auc_score(y, y_predict_lof)

auc_lof = lof_clf.fit_predict_score(x, y)

# Считаем метрику ROC AUC
print("auc_lof = ", auc_lof)

/Users/pavelmamaev/Desktop/Sait/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.8107920792079208
auc_lof =  0.8107920792079208


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import lof
from sklearn.metrics import roc_auc_score


# Создание модели
lof_clf = lof.LOF(
    contamination=0.1,
    n_neighbors=13,
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_lof = lof_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_lof = roc_auc_score(y, y_predict_lof)
print("auc_lof = ", auc_lof)

## 3️⃣👬🏻 k-nearest neighbors (kNN, Метод k-ближайших соседей)

###  🎰  TF-IDF

In [20]:
import pyod
from pyod.models import knn
from sklearn.metrics import roc_auc_score

# Создание модели
knn_clf = knn.KNN(
    contamination=0.1,
    n_neighbors=3,
    method='largest',
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_knn = knn_clf.fit_predict(x)

# Считаем метрику ROC AUC
auc_knn = knn_clf.fit_predict_score(x, y)
# auc_knn = roc_auc_score(y, y_predict_knn)
print("auc_knn = ", auc_knn)

/Users/pavelmamaev/Desktop/Sait/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.6807029702970296
auc_knn =  0.6807029702970296


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import knn
from sklearn.metrics import roc_auc_score

# Создание модели
knn_clf = knn.KNN(
    contamination=0.1,
    n_neighbors=120,
    method='largest',
    metric='l1',
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_knn = knn_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_knn = roc_auc_score(y, y_predict_knn)
print("auc_knn = ", auc_knn)

## 4️⃣📊 [Copula-Based Outlier Detection](http://www.andrew.cmu.edu/user/yuezhao2/papers/20-icdm-copod.pdf) (COPOD)
[Что такое копула?](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BF%D1%83%D0%BB%D0%B0)

###  🎰  TF-IDF

In [18]:
import pyod
from pyod.models import copod
from sklearn.metrics import roc_auc_score

# Создание модели
copod_clf = copod.COPOD(contamination=0.1)

# Тренировка и предсказание модели
# y_predict_copod = copod_clf.fit_predict(x)

# Считаем метрику ROC AUC
auc_copod = copod_clf.fit_predict_score(x, y)
# auc_copod = roc_auc_score(y, y_predict_copod)
print("auc_copod = ", auc_copod)

/Users/pavelmamaev/Desktop/Sait/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.6908316831683168
auc_copod =  0.6908316831683168


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import copod
from sklearn.metrics import roc_auc_score

# Создание модели
copod_clf = copod.COPOD(contamination=0.1)

# Тренировка и предсказание модели
y_predict_copod = copod_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_copod = roc_auc_score(y, y_predict_copod)
print("auc_copod = ", auc_copod)

## 5️⃣⌛️ AE (Автокодировщик)

In [ ]:
import pyod
from pyod.models import auto_encoder
from sklearn.metrics import roc_auc_score

# Создание модели
ae_clf = auto_encoder.AutoEncoder(
    hidden_neurons=[128, 64, 32, 64, 128],
    hidden_activation='relu',
    output_activation='sigmoid',
    optimizer='adam',
    epochs=5,
    batch_size=128,
    dropout_rate=0.4,
    l2_regularizer=0.4,
    validation_size=0.0,
    preprocessing=True,
    verbose=1,
    random_state=123,
    contamination=0.1
)

# Считаем метрику ROC AUC
ae_clf.fit(x)
y_predict_ae = ae_clf.decision_function(x)
auc_ae = roc_auc_score(y, y_predict_ae)
print("auc_ae = ", auc_ae)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 13828)             191227412 
_________________________________________________________________
dropout (Dropout)            (None, 13828)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 13828)             191227412 
_________________________________________________________________
dropout_1 (Dropout)          (None, 13828)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1770112   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8

## 6️⃣⏲ VAE (Вариационный автокодировщик)

In [ ]:
import pyod
from pyod.models import vae
from sklearn.metrics import roc_auc_score


# Создание модели
vae_clf = vae.VAE(
    contamination=0.1,
    encoder_neurons=[256, 128, 64],
    decoder_neurons=[64, 128, 256],
    latent_dim=8,
    hidden_activation='relu',
    output_activation='sigmoid',
    optimizer='adam',
    epochs=0,
    batch_size=128,
    dropout_rate=0.3,
    l2_regularizer=0.5,
    validation_size=0.0,
    preprocessing=True,
    verbose=1,
    random_state=123,
)

# Считаем метрику ROC AUC
vae_clf.fit(x)
y_predict_vae = vae_clf.predict(x)
auc_vae = roc_auc_score(y, y_predict_vae)
print("auc_vae = ", auc_vae)

In [ ]:
x[x == 1e-16] = 1e-5
x.min()